In [1]:
#import torch
#import torch.autograd as autograd

import numpy as np
from mxnetnet import MXNetNet
from pytorchnet import PytorchNet
from tfnet import TFNet
        
# bug: filter reshaping possibly causes incorrect results 
class ConvBenchmark:
    def __init__(self, **bench_args):
        convweights = bench_args["convweights"]
        self.conv_filters_ = []
        #filter_ = autograd.Variable(torch.FloatTensor([[[[-1, 1]]]])).numpy()
        for w in convweights:
            w_array = np.random.random(w).astype(np.float32)
            self.conv_filters_.append(w_array)
            #self.conv_filters_.append(filter_)
        
        
        if "poolings" in bench_args:
            self.poolings_ = bench_args["poolings"]
        else:
            self.poolings_ = False
        
        self.networks_ = {}
        self.networks_["pytorchCNN"] = PytorchNet(self.conv_filters_, self.poolings_)
        self.networks_["TF_CNN"] = TFNet(self.conv_filters_, self.poolings_)
        self.networks_["MXNet_CNN"] = MXNetNet(self.conv_filters_, self.poolings_)
        
        self.input_ = np.random.random(bench_args["inputsize"]).astype(np.float32)
        self.results_ = {}
        
    def runNetworks(self):
        # Pytorch nn.Conv2d takes input a 4D Tensor of [nSamples x nChannels x Height x Width]
        # Input is defined in Pytorch layout fashion
        for netname, network in self.networks_.items():
            #output, t = network.forward(np.copy(self.input_))
            #self.results_[netname] = {"output": output, "runtime": t}
            output, t, conv_filters = network.forward(np.copy(self.input_))
            self.results_[netname] = {"output": output, "runtime": t, "filters": conv_filters}

            print("{} took {:.3} seconds.".format(netname, t))
            
        

/kuacc/users/ccengiz17/.anaconda3/envs/dl_frameworks/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [2]:
# input is a 4D Tensor of [nSamples x nChannels x Height x Width]
inBS, inCh, inHeight, inWidth = (2,1,32,32)
inputsize = inBS, inCh, inHeight, inWidth

# Filters are 4D tensor in form [out_channels, in_channels, fil_height, fil_width]
conv1w = (6, 1, 5, 5)
conv2w = (16, 6, 5, 5)
convweights = (conv1w, conv2w)

poolings = (2,2)

benchmark = ConvBenchmark(inputsize=inputsize, convweights=convweights, poolings=poolings)

In [3]:
benchmark = ConvBenchmark(inputsize=(256,3,32,32), convweights=((512, 3, 3, 3), (1024, 512, 3, 3)), poolings=(2,2))

In [19]:
benchmark.runNetworks()

layer: 0
size of x before: torch.Size([256, 3, 32, 32])
size of filter : torch.Size([512, 3, 3, 3])
size of x after filter: torch.Size([256, 512, 30, 30])
size of x after pool: torch.Size([256, 512, 15, 15])
 
layer: 1
size of x before: torch.Size([256, 512, 15, 15])
size of filter : torch.Size([1024, 512, 3, 3])
size of x after filter: torch.Size([256, 1024, 13, 13])
size of x after pool: torch.Size([256, 1024, 6, 6])
 
pytorchCNN took 12.6 seconds.
layer: 0
size of x before: (256, 32, 32, 3)
size of filter : (3, 3, 3, 512)
size of x after filter: (256, 30, 30, 512)
size of x after pool: (256, 15, 15, 512)
 
layer: 1
size of x before: (256, 15, 15, 512)
size of filter : (3, 3, 512, 1024)
size of x after filter: (256, 13, 13, 1024)
size of x after pool: (256, 6, 6, 1024)
 
TF_CNN took 0.122 seconds.
layer: 0
size of x before: (256, 3, 32, 32)
size of filter : (512, 3, 3, 3)
size of x after filter: (256, 512, 30, 30)
size of x after pool: (256, 512, 15, 15)
 
layer: 1
size of x before: 

In [20]:
pytorch_out = benchmark.results_["pytorchCNN"]["output"]
mxnet_out = benchmark.results_["MXNet_CNN"]["output"]
tf_out = benchmark.results_["TF_CNN"]["output"]

In [21]:
print(pytorch_out.shape)
print(type(pytorch_out))

(256, 1024, 6, 6)
<class 'numpy.ndarray'>


In [22]:
print(mxnet_out.shape)
print(type(mxnet_out))

(256, 1024, 6, 6)
<class 'numpy.ndarray'>


In [23]:
tf_out = tf_out.reshape(pytorch_out.shape)
print(tf_out.shape)
print(type(tf_out))

(256, 1024, 6, 6)
<class 'numpy.ndarray'>


In [24]:
pytorch_out[0,0,0:3,0:3]

array([[ 17700.90625   ,  18105.2109375 ,  17949.94140625],
       [ 18606.359375  ,  18277.50195312,  17178.41601562],
       [ 18592.23828125,  18842.01953125,  17894.95898438]], dtype=float32)

In [25]:
mxnet_out[0,0,0:3,0:3]

array([[ 17700.90625   ,  18105.2109375 ,  17949.94140625],
       [ 18606.359375  ,  18277.50195312,  17178.41601562],
       [ 18592.23828125,  18842.01953125,  17894.95898438]], dtype=float32)

In [26]:
tf_out[0,0,0:3,0:3]

array([[ 17383.02148438,  17229.13867188,  17244.0625    ],
       [ 17229.68554688,  17093.66601562,  17100.80859375],
       [ 17220.58984375,  17143.87304688,  17195.0546875 ]], dtype=float32)

In [27]:
np.isclose(mxnet_out, pytorch_out, atol=0.001).all()

True